In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import mne
import entropy as ent
import neurokit2 as nk
from scipy.signal import welch
from nolds import lyap_r
import antropy as ant
import joblib

In [26]:
participantsInfo = pd.read_table('./ds004504/participants.tsv')
#Getting all the groups
A_sub = participantsInfo[participantsInfo["Group"] == "A"]["participant_id"].tolist()
C_sub = participantsInfo[participantsInfo["Group"] == "C"]["participant_id"].tolist()
D_sub = participantsInfo[participantsInfo["Group"] == "F"]["participant_id"].tolist()

In [27]:
freq_bands = {
    "Delta": (0.5, 4),
    "Theta": (4, 8),
    "Alpha": (8, 12),
    "Beta": (12, 30),
}

import pandas as pd
import numpy as np 

#this generates the dataframe persubject
def generate_subject_dataframe(subjects) -> pd.DataFrame:
    """
    Generates a DataFrame where each subject has 19 channels, 
    and each channel has 4 bands (Delta, Theta, Alpha, Beta).
    
    Parameters:
        num_subjects (int): Number of subjects to include in the DataFrame.
    
    Returns:
        pd.DataFrame: A DataFrame with hierarchical indexing for subjects, channels, and bands for each window.
    """
    # subjects = [f"sub-{i:03d}" for i in range(start_sub, end_sub + 1)]  # Format subject IDs as sub-XXX
    channels = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
    bands = ["Delta", "Theta", "Alpha", "Beta"]
    
    data = []
    for subject in subjects:
        for channel in channels:
            for band, val in freq_bands.items():
                value = np.random.rand()  # Example random value, replace with actual data if needed
                data.append([subject, channel, band, []])
    
    df = pd.DataFrame(data, columns=["Subject", "Channel", "Band", "Window Avg"])

    #the .setIndex is creating a multiIndex DataFrame
    return df.set_index(["Subject", "Channel", "Band"])


    # Example usage:
# df = generate_subject_dataframe(5)  # Generates a DataFrame for 5 subjects
# print(df)

In [28]:
def derivativesPath(sub): #dericatives is the preprocessed data
    return f"ds004504/{sub}/eeg/{sub}_task-eyesclosed_eeg.set"



# NUM_SUBJECTS = 2
# # Use the function inside the groups dictionary
# groups = {
#     'Alzeimers': generate_subject_dataframe(A_sub[0:NUM_SUBJECTS]),
#     'Control': generate_subject_dataframe(C_sub[0:NUM_SUBJECTS]),
#     'Dementia': generate_subject_dataframe(D_sub[0:NUM_SUBJECTS]),
# }

# NUM_SUBJECTS = 2
# # Use the function inside the groups dictionary
groups = {
    'Alzeimers': generate_subject_dataframe(A_sub),
    'Control': generate_subject_dataframe(C_sub),
    'Dementia': generate_subject_dataframe(D_sub),
}

# Print the DataFrame for 'Alzeimers'

# WINDOW_LENGTH = 1.5
# STEP_SIZE = .75 #50% sliding window
WINDOW_LENGTH = NUM_SUBJECTS
STEP_SIZE = 1.5

In [29]:
print(A_sub[0:NUM_SUBJECTS])

['sub-001', 'sub-002']


In [30]:
print(groups['Alzeimers'])
print(groups['Control'])
print(groups['Dementia'])

                      Window Avg
Subject Channel Band            
sub-001 Fp1     Delta         []
                Theta         []
                Alpha         []
                Beta          []
        Fp2     Delta         []
...                          ...
sub-036 Cz      Beta          []
        Pz      Delta         []
                Theta         []
                Alpha         []
                Beta          []

[2736 rows x 1 columns]
                      Window Avg
Subject Channel Band            
sub-037 Fp1     Delta         []
                Theta         []
                Alpha         []
                Beta          []
        Fp2     Delta         []
...                          ...
sub-065 Cz      Beta          []
        Pz      Delta         []
                Theta         []
                Alpha         []
                Beta          []

[2204 rows x 1 columns]
                      Window Avg
Subject Channel Band            
sub-066 Fp1     Delta     

# ** this is the part where we would do a for loop per group, per subject in that group **


In [31]:
#TODO: add more different types of data and more subjects ,
#Make it run faster by multiprocessing

In [32]:
import time
global_start = time.time()
#** how to run this in parallel, speciall the computer PSD for each epoch
for group in groups:
    print(f'Current Group: {group}')
    df = groups[group]

    subjects = df.index.get_level_values("Subject").unique()
    for subject in subjects:
        subject_start_time = time.time()
        raw_sub = mne.io.read_raw_eeglab(derivativesPath(subject), preload=True)
        sfreq = raw_sub.info['sfreq']
        
        # Create epochs with a fixed window size
        print(f'\tCurrent Subject: {subject}')

        start_times = np.arange(0, raw_sub.times[-1] - WINDOW_LENGTH, STEP_SIZE) 
        events = np.array([
        [int(t * sfreq), 0, 1] for t in start_times  # MNE event format
        ]) 
        
        epochs = mne.Epochs(
            raw_sub, events, event_id=1,
            tmin=0, tmax=WINDOW_LENGTH,
            baseline=None, detrend=1, preload=True, verbose=False
        )

        channel_names = raw_sub.info['ch_names']

        for x in range(len(epochs)):
            pass
            epoch = epochs[x]
            epoch_psd = epoch.compute_psd(fmin=freq_bands["Delta"][0], fmax=freq_bands["Beta"][1], verbose=False) #computing the max PSD between the wave lengths
            #print(f'Current epock: {x}') 
            for channel_name in channel_names:
                        pass
                        # print(f'\t\tCurrent channel: {channel_name}')
                        for band, (band_low, band_high) in freq_bands.items():   
                            pass
                            #print(f'Current band: {band}')
                            channel_at_freq = epoch_psd.get_data(picks=channel_name , fmin=band_low, fmax=band_high)
                            channel_at_freq_avg = channel_at_freq.mean(axis=-1)[0][0] # am I computing the average correctly ? 
                            groups[group].loc[(subject, channel_name, band), "Window Avg"].append(channel_at_freq_avg)
        print(time.time() - subject_start_time)


print(time.time() - global_start)

Current Group: Alzeimers
	Current Subject: sub-001
4.266952753067017
	Current Subject: sub-002
6.464799165725708
	Current Subject: sub-003
1.727734088897705
	Current Subject: sub-004
5.386914014816284
	Current Subject: sub-005
6.500191926956177
	Current Subject: sub-006
4.645143985748291
	Current Subject: sub-007
6.044209003448486
	Current Subject: sub-008
6.602455139160156
	Current Subject: sub-009
4.397217035293579
	Current Subject: sub-010
15.352118015289307
	Current Subject: sub-011
6.179213762283325
	Current Subject: sub-012
7.957417011260986
	Current Subject: sub-013
7.052019119262695
	Current Subject: sub-014
8.965888977050781
	Current Subject: sub-015
8.33140516281128
	Current Subject: sub-016
9.156129121780396
	Current Subject: sub-017
7.197885274887085
	Current Subject: sub-018
7.192327976226807
	Current Subject: sub-019
8.302762985229492
	Current Subject: sub-020
7.463236093521118
	Current Subject: sub-021
8.317718982696533
	Current Subject: sub-022
6.891426086425781
	Curren

In [33]:
print(groups['Control'].loc[('sub-037', 'Fp1', 'Delta'), "Window Avg"])

[np.float64(1.4678144021427485e-07), np.float64(6.665712174812052e-08), np.float64(4.7582493297868225e-08), np.float64(3.0946219263574285e-08), np.float64(3.3109242203666425e-08), np.float64(7.379265182823297e-08), np.float64(3.361127644831688e-08), np.float64(2.0652638731874993e-08), np.float64(5.827332064480202e-08), np.float64(1.1416948614023939e-07), np.float64(6.203730879057895e-08), np.float64(1.854540404091241e-07), np.float64(1.2733184683464648e-07), np.float64(8.223936275375673e-08), np.float64(2.445957493621578e-07), np.float64(3.1248633605876335e-07), np.float64(9.873463588164044e-08), np.float64(5.357837890896558e-08), np.float64(7.272950259176501e-08), np.float64(6.61903569218454e-08), np.float64(3.781186129083121e-08), np.float64(4.901592257719951e-08), np.float64(8.538237271594282e-08), np.float64(4.876682543536659e-08), np.float64(7.3790468809314e-08), np.float64(2.0107623713589414e-08), np.float64(4.6977539199220125e-08), np.float64(8.24947246697141e-08), np.float64(7.

In [34]:
# Print Alzheimer's preview
print(f"Alzheimer's preview for {A_sub[0]}")
print(groups["Alzeimers"].loc[A_sub[0]]) 

# Print Control preview
print(f"Control preview {C_sub[0]}")
print(groups["Control"].loc[C_sub[0]])  

# Print Dementia preview
print(f"Dementia preview {D_sub[0]}")
print(groups["Dementia"].loc[D_sub[0]])  

Alzheimer's preview for sub-001
                                                      Window Avg
Channel Band                                                    
Fp1     Delta  [6.176774134978669e-07, 6.73489756457922e-08, ...
        Theta  [1.2920818792198119e-08, 1.6587004384810343e-0...
        Alpha  [2.708769719447813e-09, 1.630543471696663e-09,...
        Beta   [4.09779656950798e-10, 5.400439418160383e-10, ...
Fp2     Delta  [6.575557702248297e-07, 7.399095364848234e-08,...
...                                                          ...
Cz      Beta   [2.333431893759585e-10, 3.252361810377582e-10,...
Pz      Delta  [4.8392953527460886e-08, 5.073706107893873e-08...
        Theta  [5.017421354877999e-09, 4.14754027129511e-09, ...
        Alpha  [1.5798933755440305e-09, 6.678532293799915e-10...
        Beta   [2.5266709749236074e-10, 3.460148710104552e-10...

[76 rows x 1 columns]
Control preview sub-037
                                                      Window Avg
Channel Ban

In [91]:
# ** next is PCA analysis , 
#step 1 , split the data into training and testing
#step 2 , Normalize the data
#step 3 , do PCA  ... looks really simple actually ? weird .. maybe do 2 pca's one for H to D and one for H to A